In [2]:
import gzip
import json
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

with gzip.open('xbar/1/xbar.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))
    
instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

conn=np.load('xbar/1/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)

def buildBST(array,start=0,finish=-1):
    if finish<0:
        finish = len(array)
    mid = (start + finish) // 2
    if mid-start==1:
        ltl=start
    else:
        ltl=buildBST(array,start,mid)
    
    if finish-mid==1:
        gtl=mid
    else:
        gtl=buildBST(array,mid,finish)
        
    return((array[mid],ltl,gtl))

congestion_data = np.load('xbar/1/xbar_congestion.npz')
xbst=buildBST(congestion_data['xBoundaryList'])
ybst=buildBST(congestion_data['yBoundaryList'])
demand = np.zeros(shape = [instances.shape[0],])


def getGRCIndex(x,y,xbst,ybst):
    while (type(xbst)==tuple):
        if x < xbst[0]:
            xbst=xbst[1]
        else:
            xbst=xbst[2]
            
    while (type(ybst)==tuple):
        if y < ybst[0]:
            ybst=ybst[1]
        else:
            ybst=ybst[2]
            
    return ybst, xbst


for k in range(instances.shape[0]):
    xloc = instances.iloc[k]['xloc']; yloc = instances.iloc[k]['yloc']
    i,j=getGRCIndex(xloc,yloc,xbst,ybst)
    d = 0 
    for l in list(congestion_data['layerList']): 
        lyr=list(congestion_data['layerList']).index(l)
        d += congestion_data['demand'][lyr][i][j]
    demand[k] = d
        
instances['routing_demand'] = demand

1a

In [3]:
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix
from sklearn.metrics import mean_squared_error, r2_score
import scipy.sparse as sp


seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

conn=np.load('xbar/1/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)
A = torch.FloatTensor(np.array(A.todense()))
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx
A = normalize(A)
A = torch.FloatTensor(np.array(A))


target_values = instances['routing_demand'] 

train_size = 0.7
indices = np.arange(len(target_values))
train_indices, test_indices = train_test_split(indices, train_size=train_size, random_state=seed)

scaler = StandardScaler()
features = scaler.fit_transform(instances[['xloc', 'yloc', 'cell', 'orient']]) 


features = torch.FloatTensor(features)
targets = torch.FloatTensor(target_values)

In [4]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GCN, self).__init__()
        self.num_layers = num_layers
        self.gcn_layers = nn.ModuleList([nn.Linear(input_dim, hidden_dim)])
        for _ in range(self.num_layers - 2):
            self.gcn_layers.append(nn.Linear(hidden_dim, hidden_dim))
        self.gcn_layers.append(nn.Linear(hidden_dim, output_dim))

    def forward(self, x, adj):
        for layer in self.gcn_layers[:-1]:
            x = F.relu(layer(torch.matmul(adj, x)))
        x = self.gcn_layers[-1](torch.matmul(adj, x))
        return x

input_dim = features.shape[1]
output_dim = 1  
hidden_dim = 64
num_epochs = 1000
learning_rate = 0.01



In [17]:
features.shape

torch.Size([3952, 4])

In [18]:
A.shape

torch.Size([3952, 3952])

In [7]:
for k in range(1, 6):
    model = GCN(input_dim, hidden_dim, output_dim, num_layers=k)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(3000):
        model.train()
        optimizer.zero_grad()
        outputs = model(features, A)
        loss = criterion(outputs[train_indices], targets[train_indices])
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 500 == 0:
            print(f"Layer {k}, Epoch {epoch + 1}, Loss: {loss.item()}")

    model.eval()
    with torch.no_grad():
        test_outputs = model(features, A)
        test_loss = criterion(outputs[test_indices], targets[test_indices])
        print(f"Layer {k}, Test Loss: {test_loss.item()}")

Layer 1, Epoch 500, Loss: 20.787513732910156
Layer 1, Epoch 1000, Loss: 20.491809844970703
Layer 1, Epoch 1500, Loss: 20.429702758789062
Layer 1, Epoch 2000, Loss: 20.411352157592773
Layer 1, Epoch 2500, Loss: 20.405345916748047
Layer 1, Epoch 3000, Loss: 20.402992248535156
Layer 1, Test Loss: 20.542322158813477
Layer 2, Epoch 500, Loss: 20.755247116088867
Layer 2, Epoch 1000, Loss: 20.457632064819336
Layer 2, Epoch 1500, Loss: 20.413658142089844
Layer 2, Epoch 2000, Loss: 20.406085968017578
Layer 2, Epoch 2500, Loss: 20.404157638549805
Layer 2, Epoch 3000, Loss: 20.403348922729492
Layer 2, Test Loss: 20.543048858642578
Layer 3, Epoch 500, Loss: 20.430377960205078
Layer 3, Epoch 1000, Loss: 20.407169342041016
Layer 3, Epoch 1500, Loss: 20.404335021972656
Layer 3, Epoch 2000, Loss: 20.4033203125
Layer 3, Epoch 2500, Loss: 20.40270233154297
Layer 3, Epoch 3000, Loss: 20.402206420898438
Layer 3, Test Loss: 20.541610717773438
Layer 4, Epoch 500, Loss: 20.418676376342773
Layer 4, Epoch 1000

1b

In [9]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GCN, self).__init__()
        self.num_layers = num_layers
        self.gcn_layers = nn.ModuleList([nn.Linear(input_dim, hidden_dim)])
        self.attention_weights = nn.ModuleList([nn.Linear(hidden_dim * 2, 1)])
        for _ in range(self.num_layers - 2):
            self.gcn_layers.append(nn.Linear(hidden_dim, hidden_dim))
            self.attention_weights.append(nn.Linear(hidden_dim * 2, 1))
        self.gcn_layers.append(nn.Linear(hidden_dim, output_dim))

    def forward(self, x, adj):
        for layer in self.gcn_layers[:-1]:
            x = F.relu(layer(torch.matmul(adj, x)))
        for layer_idx in range(self.num_layers - 1):
            attention_scores = self.attention_weights[layer_idx](torch.cat((x, torch.matmul(adj, x)), dim=1))
            attention_scores = F.softmax(attention_scores, dim=0)
            x = x * attention_scores + torch.matmul(adj, x) * (1 - attention_scores)
        x = self.gcn_layers[-1](torch.matmul(adj, x))
        return x


input_dim = features.shape[1]
output_dim = 1  
hidden_dim = 64
num_epochs = 1000
learning_rate = 0.01



In [10]:
for k in range(1, 6):
    model = GCN(input_dim, hidden_dim, output_dim, num_layers=k)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(3000):
        model.train()
        optimizer.zero_grad()
        outputs = model(features, A)
        loss = criterion(outputs[train_indices], targets[train_indices])
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 500 == 0:
            print(f"Layer {k}, Epoch {epoch + 1}, Loss: {loss.item()}")


    model.eval()
    with torch.no_grad():
        test_outputs = model(features, A)
        test_loss = criterion(outputs[test_indices], targets[test_indices])
        print(f"Layer {k}, Test Loss: {test_loss.item()}")

Layer 1, Epoch 500, Loss: 20.953460693359375
Layer 1, Epoch 1000, Loss: 20.461013793945312
Layer 1, Epoch 1500, Loss: 20.420631408691406
Layer 1, Epoch 2000, Loss: 20.41170310974121
Layer 1, Epoch 2500, Loss: 20.407716751098633
Layer 1, Epoch 3000, Loss: 20.405088424682617
Layer 1, Test Loss: 20.54477882385254
Layer 2, Epoch 500, Loss: 20.801067352294922
Layer 2, Epoch 1000, Loss: 20.482215881347656
Layer 2, Epoch 1500, Loss: 20.42363739013672
Layer 2, Epoch 2000, Loss: 20.410938262939453
Layer 2, Epoch 2500, Loss: 20.405954360961914
Layer 2, Epoch 3000, Loss: 20.403596878051758
Layer 2, Test Loss: 20.54331398010254
Layer 3, Epoch 500, Loss: 20.419748306274414
Layer 3, Epoch 1000, Loss: 20.406352996826172
Layer 3, Epoch 1500, Loss: 20.404333114624023
Layer 3, Epoch 2000, Loss: 20.403358459472656
Layer 3, Epoch 2500, Loss: 20.40251350402832
Layer 3, Epoch 3000, Loss: 20.402193069458008
Layer 3, Test Loss: 20.5415096282959
Layer 4, Epoch 500, Loss: 20.41258430480957
Layer 4, Epoch 1000, 

In [11]:
def leave_one_out_cross_validation(features, adjacency_matrix, demand):
    num_samples = len(features)
    test_losses = []

In [14]:
leave_one_out_cross_validation(features, A, targets)